In [ ]:
!pip install transformers torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os

# Set the correct path to your local model
model_dir = os.path.join(os.getcwd(), "tokenizador")
print(f"Loading model from: {model_dir}")

# Load model and tokenizer with local_files_only=True
try:
    tokenizer = AutoTokenizer.from_pretrained(model_dir, local_files_only=True)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, local_files_only=True)
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    # Alternative: try loading without local_files_only if files are properly structured
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)

# Leer el CSV con la ruta correcta
df = pd.read_csv("C:/Users/admin/Desktop/Proyecto 10/nlp_grupo_5_proyecto_10/data/clean/dataset_pretraining_final.csv")
texts = df["Text_clean"].astype(str).tolist()

# Procesar en lotes
batch_size = 64
all_labels = []
all_scores = []

id2label = model.config.id2label

# Detecta el tipo de las claves del id2label (int o str)
first_key = list(id2label.keys())[0]
use_int = isinstance(first_key, int)

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        preds = torch.argmax(probs, dim=1)
    if use_int:
        labels = [id2label[i] for i in preds.tolist()]
    else:
        labels = [id2label[str(i)] for i in preds.tolist()]
    scores = probs.max(dim=1).values.tolist()
    all_labels.extend(labels)
    all_scores.extend(scores)

df["sentiment_label"] = all_labels
df["sentiment_score"] = all_scores
df.to_csv("resultados_sentimiento.csv", index=False)
print("Resultados guardados en resultados_sentimiento.csv")

c:\Users\USUARIO\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'C:/Users/admin/Desktop/Proyecto 10/nlp_grupo_5_proyecto_10/tokenizador'. Use `repo_type` argument if needed.

In [6]:
print(df["sentiment_label"].value_counts())

sentiment_label
1 star     2078
5 stars     945
4 stars     297
3 stars     252
2 stars      71
Name: count, dtype: int64


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Cargar modelo y tokenizador locales
model_dir = "C:/Users/admin/Desktop/Proyecto 10/nlp_grupo_5_proyecto_10/tokenizador"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)

# Escribe aquí tus frases para probar
texts = [
    "maldito",
    "fuck you",
    "ok",
    "nice",
    "suck mi dick",
]

inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    preds = torch.argmax(probs, dim=1)

id2label = model.config.id2label
# Si las claves son string
try:
    labels = [id2label[str(i)] for i in preds.tolist()]
except:
    labels = [id2label[i] for i in preds.tolist()]

for text, label, prob in zip(texts, labels, probs.max(dim=1).values.tolist()):
    print(f"Texto: {text}\nPredicción: {label} (confianza: {prob:.2f})\n")

Texto: maldito
Predicción: 1 star (confianza: 0.84)

Texto: fuck you
Predicción: 1 star (confianza: 0.82)

Texto: ok
Predicción: 3 stars (confianza: 0.50)

Texto: nice
Predicción: 4 stars (confianza: 0.44)

Texto: suck mi dick
Predicción: 1 star (confianza: 0.36)



In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Cargar modelo y tokenizador locales
model_dir = "C:/Users/admin/Desktop/Proyecto 10/nlp_grupo_5_proyecto_10/tokenizador"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)

# Escribe aquí tus frases para probar
texts = [
    "MALDITO",
    "FUCK YOU",
    "NICE",
    "LOVE YOU",
    "SUCK MY DICK"
]

inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    preds = torch.argmax(probs, dim=1)

id2label = model.config.id2label
# Obtener las etiquetas de estrellas
try:
    labels = [id2label[str(i)] for i in preds.tolist()]
except:
    labels = [id2label[i] for i in preds.tolist()]

# Clasificar TOXICO/NO TOXICO según la estrella
toxic_pred = ["TOXICO" if int(label[0]) <= 3 else "NO TOXICO" for label in labels]

for text, label, toxic, prob in zip(texts, labels, toxic_pred, probs.max(dim=1).values.tolist()):
    print(f"Texto: {text}\nPredicción: {label} ({toxic}) (confianza: {prob:.2f})\n")

Texto: MALDITO
Predicción: 1 star (TOXICO) (confianza: 0.84)

Texto: FUCK YOU
Predicción: 1 star (TOXICO) (confianza: 0.82)

Texto: NICE
Predicción: 4 stars (NO TOXICO) (confianza: 0.44)

Texto: LOVE YOU
Predicción: 5 stars (NO TOXICO) (confianza: 0.84)

Texto: SUCK MY DICK
Predicción: 1 star (TOXICO) (confianza: 0.63)

